In [37]:
import requests
import pandas as pd
import json
import re
import datetime

In [2]:
#url = 'https://api.queryly.com/v4/search.aspx?queryly_key=62d9c40063044c14&initialized=1&&query=milei&endindex=0&batchsize=10&callback=&extendeddatafields=creator,imageresizer,promo_image&timezoneoffset'
#qry = requests.get(url).text
#json_str_match = re.search(r'JSON\.parse\(\'(.*?)\'\)', qry, re.DOTALL)
#print(json_str_match)

In [32]:
rangos = range(0,1050,30)
resultados = []

for endindex in rangos:
    url = f"https://api.queryly.com/v4/search.aspx?queryly_key=62d9c40063044c14&initialized={0}&&query=milei&endindex={endindex}&batchsize={30}&callback=&extendeddatafields=creator,imageresizer,promo_image&timezoneoffset"
    response = requests.get(url)
    content = response.text
    json_str_match = re.search(r'results \= JSON\.parse\(\'(.*?)\'\)', content, re.DOTALL)
    if json_str_match:
        json_str = json_str_match.group(1)
        json_str = json_str.replace('\\\\','\\')
        json_str = json_str.replace('\\\'','')
        resultados.append(json_str)
    else:
        data = "No JSON found in the script."
    

In [33]:
fallos = []
i = 0
for resultado in resultados:
    i += 1
    try:
        data = json.loads(resultado)
    except:
        fallos.append(i)

In [34]:
df = [pd.DataFrame(data['items']) for resultado in resultados]
dfs = pd.concat(df, ignore_index=True)


In [48]:
noticias = pd.DataFrame({
    'indice': dfs.index,
    'titulo': dfs.title,
    'fecha': pd.to_datetime(pd.to_numeric(dfs.pubdateunix), unit='s'),
    'link': dfs.link,
    'contenido': "",
    'copete': dfs.description
})

In [49]:
noticias.to_excel('03_Infobae.xlsx')